# Importing Libraries
---

In [224]:
import numpy as np
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.regression import linear_model as lm
from ipywidgets import interact
import plotly.express as px
import hvplot.pandas


# Importing + Cleaning FFM & ETF Data
-----

In [117]:
ds = web.DataReader('F-F_Research_Data_Factors_daily', 'famafrench')

In [118]:
ffm_df = ds[0]/100

In [119]:
ffm_df.index = pd.DatetimeIndex(df.index)

In [135]:
etf_tickers = ['IYK','IYC','XLP', 'IYF', 'IYH', 'IYJ', 'IYM', 'IYR', 'IYW', 'IYZ', 'IDU']

In [136]:
start = '2015-02-27'
end = '2020-01-31'
etf_df = pd.DataFrame()

In [137]:
for e in etf_tickers:
    etf_df[e]=wb.DataReader(e, data_source = 'yahoo',start = start, end = end) ['Adj Close']

In [138]:
etf_df.index = pd.DatetimeIndex(etf_df.index)

In [139]:
etf_daily_returns = etf_df.pct_change().dropna()

# Running FFM Regression
------

In [140]:
results_dict = {}
params = {}

for ticker in etf_daily_returns.columns:
    results_dict[ticker] = lm.OLS(endog=etf_daily_returns.loc[:,ticker],
                  exog=sm.add_constant(ffm_df[['Mkt-RF','SMB','HML']])).fit() 
    params[ticker] = results_dict[ticker].params

In [141]:
ffm_constants = pd.DataFrame.from_dict(params, orient='index',
                       columns=['Mkt-RF','const', 'SMB', 'HML'])

In [170]:
mkt_rf_mean = ffm_df['Mkt-RF'].mean()
smb_mean = ffm_df['SMB'].mean()
hml_mean = ffm_df['HML'].mean()
rf_mean = ffm_df['RF'].mean()

In [171]:
ffm_returns = ffm_constants['const'] + ffm_constants['Mkt-RF'] * mkt_rf_mean + ffm_constants['SMB'] * smb_mean + ffm_constants['HML'] * hml_mean + rf_mean

# Working with data
    Annualizing FFM returns
    Calculating Volatility
    Calculating Sharpe Ratio
    Cumulative Daily Returns
-----

In [144]:
ffm_returns_annual = ((ffm_returns+1)**252 -1)

In [146]:
volatility = etf_daily_returns.std()* np.sqrt(252)

In [166]:
sharpe_ratio = (ffm_returns_annual) / (volatility)

In [209]:
cumulative_daily_returns = (1 + etf_daily_returns).cumprod() - 1

In [167]:
data_df = pd.concat([ffm_returns_annual, volatility, sharpe_ratio], axis = 'columns', join = 'inner')

In [206]:
data_df.columns=([['E_Return', 'Volatility', 'Sharpe_Ratio']])

In [207]:
data_df

,E_Return,Volatility,Sharpe_Ratio
IYK,0.089343,0.119931,0.744958
IYC,0.129410,0.142046,0.911046
XLP,0.096454,0.118846,0.811588
IYF,0.132747,0.147985,0.897031
IYH,0.107067,0.152347,0.702785
IYJ,0.127766,0.149335,0.855566
IYM,0.055138,0.175118,0.314862
IYR,0.098452,0.139370,0.706410
IYW,0.221118,0.183399,1.205669
IYZ,0.040918,0.171886,0.238052


# Ticker Analysis
------

In [212]:
def daily_returns_line_plot():
    """ETFs: Line Plot"""
    daily_returns_line_plot = etf_daily_returns.hvplot.line(
                        xlabel = 'ETF',
                        ylabel = 'Volatility',
                        width = 500,
                        rot = 60,
                        ylim = (-.07, .07))
    return daily_returns_line_plot

def daily_returns_box_plot():
    """ETFs: Volatility - Box Plot"""
    daily_returns_box_plot = etf_daily_returns.hvplot.box(
                        xlabel = 'ETF',
                        ylabel = 'Volatility',
                        width = 500,
                        #text = (.02,-.02,'Volatility Band'),
                        rot = 60,
                        ylim = (-.07, .07))
    return daily_returns_box_plot

def cumulative_daily_returns_line_plot():
    """ETFs: Cumulative Returns"""
    cumulative_daily_returns_line_plot = cumulative_daily_returns.hvplot.line(
                        xlabel = 'Date',
                        width = 1000,
                        ylabel = 'Returns')
    return cumulative_daily_returns_line_plot

def sharpe_ratio_plot():
    """ETFs: Sharpe Ratio"""
    sharpe_ratio_plot = sharpe_ratio.hvplot.bar(title="Sharpe Ratios", 
                        hover_fill_color = 'Yellow',
                        xlabel = 'ETF',
                        ylabel = 'Sharpe')
    return sharpe_ratio_plot

def rolling_std_dev():
    """ETFRolling 21-Day Std. Dev."""
    rolling_std_dev = daily_returns.iloc[:,list(range(1)) + [-1]].rolling(window=21).std().hvplot.line(
                        title="Rolling 21-Day Std. Dev. of ETF")
    return rolling_std_dev

# Building Efficiency Frontier Simulation

---

# Monte Carlo Simulation
-----